In [153]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Prep Work

In [92]:
def load_and_concat_data(file_paths, column_names):
    """
    Load and concatenate data from multiple CSV files given by file_paths.

    :param file_paths: List of file paths to the CSV files.
    :param column_names: List of column names for the CSV files.

    :return pd.DataFrame: DataFrame for each feature containing raw data from the years 2023 - 2024.
    """
    df = [pd.read_csv(file_path, header=1, names=column_names) for file_path in file_paths]
    concat_data = pd.concat(df)
    concat_data.reset_index(drop=True, inplace=True)
    concat_data["timestamp"] = pd.to_datetime(concat_data["timestamp"])
    concat_data.set_index("timestamp", inplace=True)
    
    # if na values are present interpolate them based on the timestamp
    if concat_data.isna().sum().sum() > 0:
        concat_data.interpolate(method="time", inplace=True)
    
    return concat_data

### File Paths and Column Names config

In [93]:
file_paths = {
    "de_prices": ["../data/raw/de_prices_2023.csv", "../data/raw/de_prices_2024.csv"],
    "de_load": ["../data/raw/de_load_2023.csv", "../data/raw/de_load_2024.csv"],
    "de_solar_gen": ["../data/raw/de_solar_gen_2023.csv", "../data/raw/de_solar_gen_2024.csv"],
    "de_wind_gen_offshore": ["../data/raw/de_wind_gen_offshore_2023.csv", "../data/raw/de_wind_gen_offshore_2024.csv"],
    "de_wind_gen_onshore": ["../data/raw/de_wind_gen_onshore_2023.csv", "../data/raw/de_wind_gen_onshore_2024.csv"],
    "ch_load": ["../data/raw/ch_load_2023.csv", "../data/raw/ch_load_2024.csv"],
    "dk_load": ["../data/raw/dk_load_2023.csv", "../data/raw/dk_load_2024.csv"],
    "fr_load": ["../data/raw/fr_load_2023.csv", "../data/raw/fr_load_2024.csv"]
}

column_names = {
    "de_prices": ["timestamp", "de_lu_price"],
    "de_load": ["timestamp", "de_load"],
    "de_solar_gen": ["timestamp", "de_solar_gen"],
    "de_wind_gen_offshore": ["timestamp", "de_wind_gen_offshore"],
    "de_wind_gen_onshore": ["timestamp", "de_wind_gen_onshore"],
    "ch_load": ["timestamp", "ch_load"],
    "dk_load": ["timestamp", "dk_load"],
    "fr_load": ["timestamp", "fr_load"]
}

### Populate new dataframes with raw data from years 2023 and 2024

In [94]:
de_prices = load_and_concat_data(file_paths["de_prices"], column_names["de_prices"])
de_load = load_and_concat_data(file_paths["de_load"], column_names["de_load"])
de_solar_gen = load_and_concat_data(file_paths["de_solar_gen"], column_names["de_solar_gen"])
de_wind_gen_offshore = load_and_concat_data(file_paths["de_wind_gen_offshore"], column_names["de_wind_gen_offshore"])
de_wind_gen_onshore = load_and_concat_data(file_paths["de_wind_gen_onshore"], column_names["de_wind_gen_onshore"])
ch_load = load_and_concat_data(file_paths["ch_load"], column_names["ch_load"])
dk_load = load_and_concat_data(file_paths["dk_load"], column_names["dk_load"])
fr_load = load_and_concat_data(file_paths["fr_load"], column_names["fr_load"])

In [95]:
# resample the quarter hourly timeseries to hourly. see readme for affected timeseries
de_load = de_load.resample("h").mean()
de_solar_gen = de_solar_gen.resample("h").mean()
de_wind_gen_offshore = de_wind_gen_offshore.resample("h").mean()
de_wind_gen_onshore = de_wind_gen_onshore.resample("h").mean()

In [96]:
dataframes = [
    de_prices,
    de_load,
    de_solar_gen,
    de_wind_gen_offshore,
    de_wind_gen_onshore,
    ch_load,
    dk_load,
    fr_load
]

combined_df = pd.concat(dataframes, axis=1)

de_lu_price             0
de_load                 0
de_solar_gen            0
de_wind_gen_offshore    0
de_wind_gen_onshore     0
ch_load                 0
dk_load                 0
fr_load                 0
dtype: int64


# Statisical Analysis

In [ ]:
correlation_matrix = combined_df.corr(method="pearson")
correlation_matrix = correlation_matrix.drop(index=["de_lu_price"])
correlation_matrix = correlation_matrix[["de_lu_price"]].sort_values(by="de_lu_price", ascending=False)

cmap = sns.diverging_palette(20, 230, as_cmap=True)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, 
            cmap=cmap,
            annot=True,  
            square=True, 
            cbar=False,
            center=0,
            yticklabels=["DE Load", "FR Load", "DK Load", "CH Load", "DE Wind Gen Offshore", "DE Solar Gen", "DE Wind Gen Onshore"],
            xticklabels=["DE-LU Prices"]).set_title("Correlation coefficients")

plt.savefig("../reports/figures/de_lu_price_correlations.png", dpi=300)

plt.show()